## 載入資料

In [1]:
import numpy as np
import codecs
import jieba
import jieba.posseg as pseg
import re
import operator
import json
jieba.set_dictionary('./Workspace/sentiment/0616/big/jieba356726.txt')
jieba.load_userdict('./Workspace/sentiment/0616/big/jieba356726.txt')
jieba.load_userdict('./Workspace/sentiment/0616/food/fooddict2027.txt')
jieba.load_userdict('./Workspace/sentiment/0616/menu/menu50806.txt')
jieba.load_userdict('./Workspace/sentiment/0616/sentiment/negativewords.txt')
jieba.load_userdict('./Workspace/sentiment/0616/sentiment/positivewords.txt')
jieba.load_userdict('./Workspace/sentiment/0616/sentiment/negative.txt')
jieba.load_userdict('./Workspace/sentiment/0616/sentiment/more.txt')
jieba.load_userdict('./Workspace/sentiment/0616/sentiment/question.txt')
# 負面
negdict = []
# 正面
posdict = []
# 否定
nodict = []
# 程度
plusdict = []
# 不肯定
question = []
with codecs.open('./expand/negativeWordMerge.txt', 'r', 'utf-8') as f:
    for w in f.readlines():
        negdict.append(w.split()[0])
with codecs.open('./expand/positiveWordMerge.txt', 'r', 'utf-8') as f:
    for w in f.readlines():
        posdict.append(w.split()[0])
with codecs.open('./expand/inverseWordMerge.txt', 'r', 'utf-8') as f:
    for w in f.readlines():
        nodict.append(w.split()[0])
with codecs.open('./expand/degreeWordMerge.txt', 'r', 'utf-8') as f:
    for w in f.readlines():
        plusdict.append(w.split()[0])
with codecs.open('./Workspace/sentiment/0616/sentiment/question.txt', 'r', 'utf-8') as f:
    for w in f.readlines():
        question.append(w.split()[0])

Building prefix dict from /Users/fan/anaconda/bin/Workspace/sentiment/0616/big/jieba356726.txt ...
DEBUG:jieba:Building prefix dict from /Users/fan/anaconda/bin/Workspace/sentiment/0616/big/jieba356726.txt ...
Loading model from cache /var/folders/ws/qq63c39d43l_20sybqcwcf900000gn/T/jieba.u60b07a305259704c4cd827282b04b44e.cache
DEBUG:jieba:Loading model from cache /var/folders/ws/qq63c39d43l_20sybqcwcf900000gn/T/jieba.u60b07a305259704c4cd827282b04b44e.cache
Loading model cost 0.435 seconds.
DEBUG:jieba:Loading model cost 0.435 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


## 建立情緒分析類別

In [4]:
class SentimentAnalysis():
    
    '''建立instance時即讀取字典'''
    def __init__(self, negdict, posdict, nodict, plusdict, question, bid, aid):
        self.score = 0
        self._id = bid
        self.author_id = aid
        self.positiveSubject = []
        self.negativeSubject = []
        self.negdict = negdict
        self.posdict = posdict
        self.nodict = nodict
        self.plusdict = plusdict
        self.question = question
    
    ''' 
        [情緒分數]
        
        計算情緒分數時，先考慮情緒詞正負，再考慮程度與極性      
        例如：好吃的義大利麵 => +1
             不好吃的義大利麵 => -1
             很好吃的義大利麵 => +2
             很不好吃的義大利麵 => -2
             
        考量大部分人傾向用詞委婉，情緒詞正負方向採用不同的處理方式
        對於負評的認定較為寬鬆，而對於正評的認定較為嚴格      
        例如：這個漢堡不算難吃 => 0
             這個漢堡不算好吃 => -1
             
        當否定詞與情緒詞中間夾雜程度用語時，皆判定為中性偏負           
        例如：這個漢堡不算非常難吃 => -0.5
             這個漢堡不算非常好吃 => -0.5
    '''
    def checkDegree(self, sdrange, positive=True):
        # 定義初始極性
        if positive:
            pollar = 1
        else:
            pollar = -1
        # 定義初始程度
        degree = 1
        # 在指定範圍內撈否定詞、程度詞
        for i in xrange(len(sdrange)):
            st = sdrange[i]
            # 標記程度用語是否出現在否定詞之前
            flag = False
            # 標記程度用語是否穿插在否定詞與情緒詞之間
            ambiguous = False
            # 標記是否為負面情緒詞前面含負面詞而不含程度詞
            excp = False
            # 遇到否定用語就改變極性方向
            if st in self.nodict:
                pollar *= -1
            # 若含程度用語
            if st in self.plusdict:
                flag = True
                # 在程度用語前面的範圍找否定詞
                for j in xrange(i):
                    if sdrange[j] in self.nodict:
                        flag = False
                        # 找到代表程度用語穿插在否定詞語情緒詞之間，予以標記
                        ambiguous = True
                        break
                # 若程度用語出現在否定詞之前則乘數設為2
                if flag:
                    degree = 2
                # 若程度用語穿插在否定詞語情緒詞之間，正向詞乘數為-0.5，負向詞乘數為0.5，最後各自乘上極性後分數同為-0.5
                elif ambiguous:
                    if pollar == 1 :
                        degree = -0.5
                    else:
                        degree = 0.5
            # 負向詞遇到否定詞，若沒有程度詞修飾，直接認定為中性
            elif not positive:
                for x in xrange(len(sdrange)):
                    if sdrange[x] in self.nodict:
                        excp = True
                        break
        if excp:
            return 0
        else:
            return pollar * degree
        
    '''
        [情緒分數加總]
        
        找到情緒詞後往前找4個詞，把4個詞丟進checkDegree判斷規則，回傳該情緒詞的分數
        每次計算完分數，判斷正負號，再執行getSubject去抓取正負主題詞，結果存入self.positiveSubject/self.negativeSubject
        並把每次的計算分數累加存入self.score
        
    '''
    def sentimentScore(self, sd):
        #jieba版
        #words = [w.word for w in sd]
        #dict版
        words = [w['word'] for w in sd]
        # 含這些髒東西的句子之後會被跳過
        #pattern = u'文章|本文'
        # 搜尋情緒詞時用倒著找的方式，便於之後主題詞的抓取
        for i in xrange(len(words)-1,-1,-1):
            p = 0
            possbj = None
            negsbj = None
            #遇到髒東西或者是出現使整個句子情緒方向不定的字眼時直接跳過 ex: 見仁見智
            #if re.search(pattern, ''.join(words)) or words[i] in self.question:
            #    break
            # 定義選取範圍
            if i > 3:
                n = 4
            else:
                n = i
            # 若出現在負面詞
            if words[i] in self.negdict:
                if n > 0:
                    p = self.checkDegree(words[i-n:i],positive=False)
                    self.score += p
                else:
                    p = 1
                    self.score -= p
            # 若出現在正面詞
            elif words[i] in self.posdict:
                if n > 0:
                    p = self.checkDegree(words[i-n:i],positive=True)
                    self.score += p
                else:
                    p = 1
                    self.score += p
            # 分別抓取正負情緒詞所對應的主題詞，存入positiveSubject/negativeSubject
            # 主題詞只保留2個字以上的
            if p > 0:
                possbj = self.getSubject(i, sd)
                if possbj and len(possbj)>1:
                    self.positiveSubject.append((possbj, words[i]))
            elif p < 0:
                negsbj = self.getSubject(i, sd)
                if negsbj and len(negsbj)>1:
                    self.negativeSubject.append((negsbj,words[i]))
                    
    '''
        [主題詞抓取原則]
        
        找到情緒詞後，先往後找名詞，並確認是否已在清單中
        若找不到或者是找到的詞已在清單中，則再往前找名詞
    '''   
    def getSubject(self, index, sd):
        subject = None
        end = len(sd)-1
        rindex = index
        lindex = index
        # 是否繼續往前找
        keepgoing = True
        # 把目前已有的positiveSubject/negativeSubject合併
        temp = map(operator.itemgetter(0), self.positiveSubject) + map(operator.itemgetter(0), self.negativeSubject)
        # 往右找
        rcount = 0
        while(rindex < end):
            rcount += 1
            if rcount > 5:
                break
            rindex += 1
            # 詞性 jieba
            #flag = sd[rindex].flag
            flag = sd[rindex]['flag']
            # 詞 jieba
            #word = sd[rindex].word
            word = sd[rindex]['word']
            # 只要詞性包含n就抓下來
            if 'n' in flag:
                if word not in temp:
                    subject = word
                    keepgoing = False
                break
        # 若往右找沒找到或者找到的已在清單中，則往前找
        lcount = 0
        if keepgoing:
            while(lindex > 0):
                if lcount > 5:
                    break
                lindex -= 1
                #flag = sd[rindex].flag
                flag = sd[lindex]['flag']
                #word = sd[rindex].word
                word = sd[lindex]['word']
                if 'n' in flag:
                    if word not in temp:
                        subject = word
                    break
        return subject

## 測試文章

In [3]:
import json
with open('./Workspace/jieba_cut.json', 'r') as f:
    text = json.load(f)

In [4]:
simpleBlog = {}
for ele in text:
    blog = {}
    blog['_id'] = ele
    blog['author_id'] = text[ele]['author_id']
    blog['article'] = [cut['word'] for cut in text[ele]['article']]
    simpleBlog[ele] = blog
with open('./Workspace/simpleBlog.json', 'w') as f:
    json.dump(simpleBlog,f)

TypeError: dump() takes at least 2 arguments (1 given)

In [5]:
with open('./Workspace/simpleBlog.json', 'w') as f:
    json.dump(simpleBlog,f)

In [31]:
import random
ri = random.randint(0,len(text))
a0 = text.values()[ri]['article']
sa0 = SentimentAnalysis(negdict, posdict, nodict, plusdict, question, 'b1', 'a1')
sa0.sentimentScore(a0)

print '情緒總分：%s' % str(sa0.score)

for ele in sa0.positiveSubject:
    print 'positive', ele[0], ele[1]
for ele in sa0.negativeSubject:
    print 'negative', ele[0], ele[1]
#for w in a0:
#    print w['word']

情緒總分：43
positive http 讚
positive justpasta 值得
positive 捷運 不錯
positive 整體 口感
positive 廚師 用心
positive 蒜味 很夠
positive 義大利麵 讚不絕口
positive 友人 好吃
positive 入口 好吃
positive 反應 不錯
positive 雙拼 推薦
positive 俗氣 用心
positive 蕃茄醬汁 不賴
positive 賣相 不錯
positive 多汁 外酥內軟
positive 炸雞塊 水準之上
positive 強力 推薦
positive 薯條 好吃
positive 效果 加分
positive 蕃茄醬 自製
positive 工廠 自製
positive 肉醬 自製
positive 凱薩沙拉 驚訝
positive 起司 不可思議
positive 價格 好喝
positive 冰咖啡 特別
negative 口感 彈牙
negative 酸味 單調


In [32]:
for w in a0:
    print w['word']

經
部落
客
友人
h
介紹
有幸
前來
這家
已經
開
了
八年
的
justpasta
必須
很
老實
的
說
其實
出發
前
我
以為
是
小店
口味
可能
普普
但
每
一樣
都
吃
過
以後
發現
小店
這
詞
沒錯
可是
味道
還
不賴
耶
絕非
普普
之輩
店
內
菜單
若
看不清楚
上圖
可
點選
上圖
至
flickr
再
用
右鍵
選
更
大
的
圖
來看
席間
友
人們
與
我
點
的
飲料
像是
可爾必斯
水果冰沙
冰咖啡
等
表現
多
屬
中規中矩
不過
我
倒
是
覺得
冰咖啡
特別
的
好喝
且
價格
很
平易近人
蠻
推
的
雞肉
大
凱薩沙拉
友人
a
的
味蕾
超強
吃
出來
上面
灑
有
什麼
名店
的
某種
起司
實在
是
太
不可思議
了
我
完全
不
知道
那
是
什麼
啊啊啊
而且
美麗大方
的
老闆娘
有
證實
友人
a
的
說法
確實
沒錯
所以
這
凱薩沙拉
好吃
之餘
我
實在
是
驚訝
友人
a
的
味覺
與
見識
之
廣
炸薯條
有
兩種
沾
醬
自製
的
蕃茄醬
與
起司
肉醬
自製
蕃茄醬
不是
我
第一次
吃到
之前
在
也
有
吃
過
類似
的
味道
真是
好
自製
的
就是
與
工廠
生產
的
有
很大
的
不同
啊
justpasta
的
也
不
例外
所以
薯條
來
沾
這個
自製
蕃茄醬
真的
是
頗
有
加分
效果
另外
那個
起司
肉醬
也
是
搭配
薯條
之用
同樣
的
也
是
好吃
而且
薯條
與
肉醬
的
搭配
不
常見
真的
是
要
強力
推薦
炸雞塊
水準之上
的
表現
外酥內軟
又
多汁
搭配
一旁
的
沾
醬
也
很
不錯
友人
老公
點
的
濃湯
不
知道
味道
如何
看起來
賣相
不錯
友人
b
點
的
蕃茄海鮮義大利麵
也
是
我們
之中
唯一
採
單點
菜色
的
人
其他人
包括
我
在內
都
是
點
雙拼
雙拼
馬上
介紹
我
有
吃
一隻
蝦子
嚐
味道
還
不賴
蕃茄醬汁
有
用心
不是
那種
很
俗氣
的
單調
酸味
而且
我
記得
友人
說
麵
身
還
蠻
彈牙
所以
這
道
很
值得
推薦
接著
就
都
是
雙拼
的
照片
雙拼
就是
可以
指定
店
內
任意
兩種
主菜
份量
會
減半
然後
組合成
一份
不管
你
選
哪種
主菜
雙拼
的
價錢
都
是

In [53]:
subject = {}
for ele in text: 
    article = text[ele]['article']
    SA = SentimentAnalysis(negdict, posdict, nodict, plusdict, question, ele, text[ele]['author_id'])
    SA.sentimentScore(article)
    obj2dict = SA.__dict__
    [obj2dict.pop(key,None) for key in ['negdict','posdict','nodict','plusdict', 'question']]
    subject[ele] = obj2dict

## 丟檔案

In [55]:
with open('./Workspace/data/subject.json', 'w') as f:
    json.dump(subject,f)